# Context Editing & Memory for Long-Running Agents

Learn how to build AI agents that learn and improve across conversations using Claude's memory tool and context editing capabilities.

## Table of Contents

1. [Introduction: Why Memory Matters](#introduction)
2. [Use Cases](#use-cases)
3. [Quick Start Examples](#quick-start)
4. [How It Works](#how-it-works)
5. [Code Review Assistant Demo](#demo)
6. [Real-World Applications](#real-world)
7. [Best Practices](#best-practices)

## Setup

### For VSCode Users

```bash
# 1. Create virtual environment
python -m venv .venv

# 2. Activate it
source .venv/bin/activate  # macOS/Linux
# or: .venv\Scripts\activate  # Windows

# 3. Install dependencies
pip install -r requirements.txt

# 4. In VSCode: Select .venv as kernel (top right)
```

### API Key

```bash
cp .env.example .env
# Edit .env and add your ANTHROPIC_API_KEY
```

Get your API key from: https://console.anthropic.com/


## 1. Introduction: Why Memory Matters {#introduction}

This cookbook demonstrates practical implementations of the context engineering patterns described in [Effective context engineering for AI agents](https://www.anthropic.com/engineering/effective-context-engineering-for-ai-agents). That post covers why context is a finite resource, how attention budgets work, and strategies for building effective agents—the techniques you'll see in action here.

### The Problem

Large language models have finite context windows (200k tokens for Claude 4). While this seems large, several challenges emerge:

- **Context limits**: Long conversations or complex tasks can exceed available context
- **Computational cost**: Processing large contexts is expensive - attention mechanisms scale quadratically
- **Repeated patterns**: Similar tasks across conversations require re-explaining context every time
- **Information loss**: When context fills up, earlier important information gets lost

### The Solution

Claude 4 models introduce powerful context management capabilities:

1. **Memory Tool** (`memory_20250818`): Enables cross-conversation learning
   - Claude can write down what it learns for future reference
   - File-based system under `/memories` directory
   - Client-side implementation gives you full control

2. **Context Editing**: Automatically manages context with two strategies:
   - **Tool use clearing** (`clear_tool_uses_20250919`): Clears old tool results when context grows large
   - **Thinking management** (`clear_thinking_20251015`): Manages extended thinking blocks (requires thinking enabled)
   - Configurable triggers and retention policies

### The Benefit

Build AI agents that **get better at your specific tasks over time**:

- **Session 1**: Claude solves a problem, writes down the pattern
- **Session 2**: Claude applies the learned pattern immediately (faster!)
- **Long sessions**: Context editing keeps conversations manageable

Think of it as giving Claude a notebook to take notes and refer back to - just like humans do.

### What You'll Learn

By the end of this cookbook, you will be able to:
- **Implement** the memory tool for cross-conversation learning
- **Configure** context editing to manage long-running sessions
- **Apply** best practices for memory security and organization

## 2. Use Cases {#use-cases}

Memory and context management enable powerful new workflows:

### 🔍 Code Review Assistant
- Learns debugging patterns from past reviews
- Recognizes similar bugs instantly in future sessions
- Builds team-specific code quality knowledge
- **Production ready**: Integrate with [claude-code-action](https://github.com/anthropics/claude-code-action) for GitHub PR reviews

### 📚 Research Assistant
- Accumulates knowledge on topics over multiple sessions
- Connects insights across different research threads
- Maintains bibliography and source tracking

### 💬 Customer Support Bot
- Learns user preferences and communication style
- Remembers common issues and solutions
- Builds product knowledge base from interactions

### 📊 Data Analysis Helper
- Remembers dataset patterns and anomalies
- Stores analysis techniques that work well
- Builds domain-specific insights over time

**Supported Models**: Claude Opus 4.1 (`claude-opus-4-1`), Claude Opus 4 (`claude-opus-4`), Claude Sonnet 4.5 (`claude-sonnet-4-5`), Claude Sonnet 4 (`claude-sonnet-4`), and Claude Haiku 4.5 (`claude-haiku-4-5`)

**This cookbook focuses on the Code Review Assistant** as it clearly demonstrates both memory (learning patterns) and context editing (handling long reviews).

## 3. Quick Start Examples {#quick-start}

Let's see memory and context management in action with simple examples.

### Setup

First, install dependencies and configure your environment:

In [1]:
%%capture
# Install required packages
# Option 1: From requirements.txt
# %pip install -q -r requirements.txt

# Option 2: Direct install
%pip install -q anthropic python-dotenv ipykernel

**⚠️ Important**: Create a `.env` file in this directory:

```bash
# Copy .env.example to .env and add your API key
cp .env.example .env
```

Then edit `.env` to add your Anthropic API key from https://console.anthropic.com/

In [2]:
import os
from typing import Any, cast

from anthropic import Anthropic
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Model configuration - use alias for automatic updates
MODEL = "claude-sonnet-4-5"  # Can override via ANTHROPIC_MODEL env var
if os.getenv("ANTHROPIC_MODEL"):
    MODEL = os.getenv("ANTHROPIC_MODEL")

API_KEY = os.getenv("ANTHROPIC_API_KEY")
if not API_KEY:
    raise ValueError(
        "ANTHROPIC_API_KEY not found. "
        "Copy .env.example to .env and add your API key."
    )

client = Anthropic(api_key=API_KEY)

print("✓ API key loaded")
print(f"✓ Using model: {MODEL}")

✓ API key loaded
✓ Using model: claude-sonnet-4-5


### Example 1: Basic Memory Usage

Let's see Claude use memory to store information for future reference.

**Helper Functions**

These examples use helper functions from `demo_helpers.py`:

- **`run_conversation_loop()`**: Handles the API conversation loop
  - Calls Claude's API with memory tool enabled
  - Executes tool uses (memory operations)
  - Continues until Claude stops using tools
  - Returns the final response

- **`run_conversation_turn()`**: Single turn (used in Example 3)
  - Same as above but returns after one API call
  - Useful when you need fine-grained control

- **`print_context_management_info()`**: Displays context clearing stats
  - Shows tokens saved, tool uses cleared
  - Helps visualize when context editing triggers

**⚠️ Note on Memory Clearing**

The following cell clears all memory files to provide a clean slate for this demonstration. This is useful for running the notebook multiple times to see consistent results.

**In production applications**, you should carefully consider whether to clear all memory, as it permanently removes learned patterns. Consider using selective deletion or organizing memory into project-specific directories instead.

In [3]:
# Import helper functions
from memory_demo.demo_helpers import run_conversation_loop, run_conversation_turn, print_context_management_info
from memory_tool import MemoryToolHandler

# Initialize
client = Anthropic(api_key=API_KEY)
memory = MemoryToolHandler(base_path="./demo_memory")

# Clear any existing memories to start fresh
print("🧹 Clearing previous memories...")
memory.clear_all_memory()
print("✓ Memory cleared\n")

# Load example code with a race condition bug
with open("memory_demo/sample_code/web_scraper_v1.py", "r") as f:
    code_to_review = f.read()

messages = [
    {
        "role": "user",
        "content": f"I'm reviewing a multi-threaded web scraper that sometimes returns fewer results than expected. The count is inconsistent across runs. Can you find the issue?\n\n```python\n{code_to_review}\n```"
    }
]

print("=" * 60)
print("📝 SESSION 1: Learning from a bug")
print("=" * 60)

# Run conversation loop
response = run_conversation_loop(
    client=client,
    model=MODEL,
    messages=messages,
    memory_handler=memory,
    system="You are a code reviewer.",
    max_tokens=2048,
    max_turns=5,
    verbose=True
)

print("\n" + "=" * 60)
print("✅ Session 1 complete!")
print("=" * 60)

🧹 Clearing previous memories...
✓ Memory cleared

📝 SESSION 1: Learning from a bug

🔄 Turn 1:
💬 Claude: I'll review this code for the race condition issues. Let me start by checking my memory.

  🔧 Memory tool: view /memories
  ✓ Result: Directory: /memories
(empty)

🔄 Turn 2:
  🔧 Memory tool: create /memories/review_progress.md
  ✓ Result: File created successfully at /memories/review_progress.md

🔄 Turn 3:
💬 Claude: ## Code Review: Multi-threaded Web Scraper

### **Critical Issues Found** 🔴

I've identified the race condition bugs causing inconsistent results. Here's my detailed analysis:

---

### **1. Primary Race Condition: Unsafe List Operations** 
**Severity: CRITICAL**

**Location:** Lines in `scrape_urls()` method
```python
self.results.append(result)  # RACE CONDITION
self.failed_urls.append(result["url"])  # RACE CONDITION
```

**Problem:**
- Python's `list.append()` is **NOT thread-safe** despite the GIL (Global Interpreter Lock)
- While the GIL prevents true parallelism, i

**What happened?**

1. Claude checked its memory (empty on first run)
2. Identified the bug: **race condition** - multiple threads modifying shared state (`self.results` and `self.failed_urls`) without synchronization
3. Stored the concurrency pattern in memory for future reference

Now let's see the magic - Claude applying this learned pattern in a **new conversation**:

### Example 2: Cross-Conversation Learning

Start a completely new conversation - memory persists!

In [4]:
# NEW conversation (empty messages)
# Load API client code with similar concurrency issue
with open("memory_demo/sample_code/api_client_v1.py", "r") as f:
    code_to_review = f.read()

messages = [
    {
        "role": "user",
        "content": f"Review this API client code:\n\n```python\n{code_to_review}\n```"
    }
]

print("=" * 60)
print("🚀 SESSION 2: Applying learned pattern")
print("=" * 60)

# Run conversation loop
response = run_conversation_loop(
    client=client,
    model=MODEL,
    messages=messages,
    memory_handler=memory,
    system="You are a code reviewer.",
    max_tokens=2048,
    max_turns=5,
    verbose=True
)

print("\n" + "=" * 60)
print("✅ Session 2 complete!")
print("=" * 60)

🚀 SESSION 2: Applying learned pattern

🔄 Turn 1:
  🔧 Memory tool: view /memories
  ✓ Result: Directory: /memories
- review_progress.md

🔄 Turn 2:
  🔧 Memory tool: view /memories/review_progress.md
  ✓ Result:    1: # Code Review: Multi-threaded Web Scraper
   2: 
   3: ## Task
   4: Revie...

🔄 Turn 3:
  🔧 Memory tool: str_replace /memories/review_progress.md
  ✓ Result: File /memories/review_progress.md has been edited successfully

🔄 Turn 4:
💬 Claude: Now let me provide a comprehensive code review:

## Code Review: AsyncAPIClient

### 🔴 **Critical Issues**

#### 1. **Race Conditions on Shared State** (High Severity)
**Location**: Lines 34-41 in `fetch_all()` method

```python
# PROBLEM CODE:
if "error" in result:
    self.error_count += 1  # Not atomic!
else:
    self.responses.append(result)  # Not thread-safe in async context!
```

**Issue**: While Python's GIL prevents true concurrent execution in threads, **async code CAN still have race conditions** because:
- Coroutines can be 

**Notice the difference:**

- Claude **immediately checked memory** and found the thread-safety/concurrency pattern
- Recognized the similar issue in async code **instantly** without re-learning
- Response was **faster** because it applied stored knowledge about shared mutable state

This is **cross-conversation learning** in action!

### Example 3: Context Clearing While Preserving Memory

What happens during a **long review session** with many code files?

- Context fills up with tool results from previous reviews
- But memory (learned patterns) must persist!

Let's trigger **context editing** to see how Claude manages this automatically.

In [5]:
# Configure context management with BOTH clearing strategies
# Low thresholds for demo - in production use 30-40k tokens
CONTEXT_MANAGEMENT = {
    "edits": [
        # Thinking management MUST come first when combining strategies
        {
            "type": "clear_thinking_20251015",
            "keep": {"type": "thinking_turns", "value": 1}  # Keep only last turn's thinking
        },
        {
            "type": "clear_tool_uses_20250919",
            "trigger": {"type": "input_tokens", "value": 5000},  # Low threshold for demo
            "keep": {"type": "tool_uses", "value": 2},  # Keep last 2 tool uses
            "clear_at_least": {"type": "input_tokens", "value": 2000}
        }
    ]
}

# Extended thinking config (required for clear_thinking strategy)
THINKING = {
    "type": "enabled",
    "budget_tokens": 1024  # Budget for thinking per turn
}

# Continue from previous session - memory persists!
print("=" * 60)
print("📚 SESSION 3: Long review session with context clearing")
print("=" * 60)
print()

# Review 1: Data processor (larger file)
with open("memory_demo/sample_code/data_processor_v1.py", "r") as f:
    data_processor_code = f.read()

messages.extend([
    {
        "role": "user",
        "content": f"Review this data processor:\n\n```python\n{data_processor_code}\n```"
    }
])

print("📝 Review 1: Data processor")
response = run_conversation_turn(
    client=client,
    model=MODEL,
    messages=messages,
    memory_handler=memory,
    system="You are a code reviewer.",
    context_management=CONTEXT_MANAGEMENT,
    thinking=THINKING,
    max_tokens=4096,
    verbose=True
)

# Add response to messages
messages.append({"role": "assistant", "content": response[1]})
if response[2]:
    messages.append({"role": "user", "content": response[2]})

print(f"  📊 Input tokens: {response[0].usage.input_tokens:,}")
context_cleared, saved = print_context_management_info(response[0])
print()

# Review 2: SQL code
with open("memory_demo/sample_code/sql_query_builder.py", "r") as f:
    sql_code = f.read()

messages.extend([
    {
        "role": "user",
        "content": f"Review this SQL query builder:\n\n```python\n{sql_code}\n```"
    }
])

print("📝 Review 2: SQL query builder")
response = run_conversation_turn(
    client=client,
    model=MODEL,
    messages=messages,
    memory_handler=memory,
    system="You are a code reviewer.",
    context_management=CONTEXT_MANAGEMENT,
    thinking=THINKING,
    max_tokens=4096,
    verbose=True
)

messages.append({"role": "assistant", "content": response[1]})
if response[2]:
    messages.append({"role": "user", "content": response[2]})

print(f"  📊 Input tokens: {response[0].usage.input_tokens:,}")
context_cleared, saved = print_context_management_info(response[0])
print()

# Review 3: Add one more review to ensure we trigger clearing
with open("memory_demo/sample_code/web_scraper_v1.py", "r") as f:
    scraper_code = f.read()

messages.extend([
    {
        "role": "user",
        "content": f"Quick check - any issues here?\n\n```python\n{scraper_code}\n```"
    }
])

print("📝 Review 3: Web scraper (should trigger clearing)")
response = run_conversation_turn(
    client=client,
    model=MODEL,
    messages=messages,
    memory_handler=memory,
    system="You are a code reviewer.",
    context_management=CONTEXT_MANAGEMENT,
    thinking=THINKING,
    max_tokens=4096,
    verbose=True
)

messages.append({"role": "assistant", "content": response[1]})
if response[2]:
    messages.append({"role": "user", "content": response[2]})

print(f"  📊 Input tokens: {response[0].usage.input_tokens:,}")
context_cleared, saved = print_context_management_info(response[0])

print()
print("=" * 60)
print("✅ Session 3 complete!")
print("=" * 60)

📚 SESSION 3: Long review session with context clearing

📝 Review 1: Data processor
🧠 Thinking: Let me review this data processor code. I should first check my memory, then provide a comprehensive...
  🔧 Memory tool: view /memories
  ✓ Result: Directory: /memories
- review_progress.md
  📊 Input tokens: 6,651
  ℹ️  Context below threshold - no clearing triggered

📝 Review 2: SQL query builder
🧠 Thinking: The user is now asking me to review a SQL query builder with SQL injection vulnerabilities. Let me c...
  🔧 Memory tool: str_replace /memories/review_progress.md
  ✓ Result: File /memories/review_progress.md has been edited successfully
  📊 Input tokens: 7,740
  ✂️  Context editing triggered!
      • Cleared 0 tool uses
      • Saved 38 tokens
      • After clearing: 7,740 tokens

📝 Review 3: Web scraper (should trigger clearing)
🧠 Thinking: The user is asking for a quick check on this web scraper code. Let me identify the issues:

1. **Rac...
  🔧 Memory tool: str_replace /memories/revie

**What just happened?**

As context grew during multiple reviews with extended thinking enabled, context editing was applied:
1. **Thinking blocks cleared** - Old thinking from previous turns removed first
2. **Tool results cleared** - Old memory tool results removed when threshold exceeded
3. **Memory files intact** - Claude can still query learned patterns
4. **Token usage managed** - Saved tokens from both thinking and tool results

This demonstrates the key benefit:
- **Short-term memory** (conversation context + thinking) → Cleared to save space
- **Long-term memory** (stored patterns) → Persists across sessions

Let's verify memory survived the clearing:

In [6]:
# Verify memory persists after context clearing
import os

print("📂 Memory files in demo_memory/:")
print()

for root, dirs, files in os.walk("./demo_memory"):
    # Calculate relative path for display
    level = root.replace("./demo_memory", "").count(os.sep)
    indent = "  " * level
    folder_name = os.path.basename(root) or "demo_memory"
    print(f"{indent}{folder_name}/")
    
    sub_indent = "  " * (level + 1)
    for file in files:
        file_path = os.path.join(root, file)
        size = os.path.getsize(file_path)
        print(f"{sub_indent}├── {file} ({size} bytes)")

print()
print("✅ All learned patterns preserved despite context clearing!")

📂 Memory files in demo_memory/:

demo_memory/
  memories/
    ├── review_progress.md (255 bytes)

✅ All learned patterns preserved despite context clearing!


## 4. How It Works {#how-it-works}

### Memory Tool Architecture

The memory tool is **client-side** - you control the storage. Claude makes tool calls, your application executes them.

#### Memory Tool Commands

| Command | Description | Example |
|---------|-------------|---------|
| `view` | Show directory or file contents | `{"command": "view", "path": "/memories"}` |
| `create` | Create or overwrite a file | `{"command": "create", "path": "/memories/notes.md", "file_text": "..."}` |
| `str_replace` | Replace text in a file | `{"command": "str_replace", "path": "...", "old_str": "...", "new_str": "..."}` |
| `insert` | Insert text at line number | `{"command": "insert", "path": "...", "insert_line": 2, "insert_text": "..."}` |
| `delete` | Delete a file or directory | `{"command": "delete", "path": "/memories/old.txt"}` |
| `rename` | Rename or move a file | `{"command": "rename", "old_path": "...", "new_path": "..."}` |

See `memory_tool.py` for the complete implementation with path validation and security measures.

### Thinking Management (`clear_thinking_20251015`)

When using extended thinking, thinking blocks accumulate and consume tokens. The `clear_thinking` strategy manages these automatically.

**Important**: This strategy requires `thinking` to be enabled in your API call.

**API Call Pattern** (with extended thinking enabled):

```python
response = client.beta.messages.create(
    betas=["context-management-2025-06-27"],  # Required beta flag
    model="claude-sonnet-4-5",
    messages=messages,
    tools=[{"type": "memory_20250818", "name": "memory"}],
    thinking={"type": "enabled", "budget_tokens": 10000},  # Enable thinking
    context_management={  # Context editing config
        "edits": [
            {
                "type": "clear_thinking_20251015",
                "keep": {"type": "thinking_turns", "value": 1}  # Keep last turn only
            },
            {
                "type": "clear_tool_uses_20250919",
                "trigger": {"type": "input_tokens", "value": 35000},
                "keep": {"type": "tool_uses", "value": 5}
            }
        ]
    },
    max_tokens=2048
)
```

**Key points:**
- `clear_thinking` must come **first** when combining strategies
- Requires extended thinking to be enabled (`thinking={"type": "enabled", ...}`)
- Use `"keep": "all"` to preserve all thinking blocks for maximum cache hits
- Trigger is optional for thinking (clears based on `keep` value)

### Understanding the Demo Code

Key implementation details from `code_review_demo.py`:

```python
class CodeReviewAssistant:
    def __init__(self, memory_storage_path="./memory_storage"):
        self.client = Anthropic(api_key=API_KEY)
        self.memory_handler = MemoryToolHandler(base_path=memory_storage_path)
        self.messages = []
    
    def review_code(self, code, filename, description=""):
        # 1. Add user message
        self.messages.append({...})
        
        # 2. Conversation loop with tool execution
        while True:
            response = self.client.beta.messages.create(
                model=MODEL,
                system=self._create_system_prompt(),
                messages=self.messages,
                tools=[{"type": "memory_20250818", "name": "memory"}],
                betas=["context-management-2025-06-27"],
                context_management=CONTEXT_MANAGEMENT
            )
            
            # 3. Execute tool uses
            tool_results = []
            for content in response.content:
                if content.type == "tool_use":
                    result = self._execute_tool_use(content)
                    tool_results.append({...})
            
            # 4. Continue if there are tool uses, otherwise done
            if tool_results:
                self.messages.append({"role": "user", "content": tool_results})
            else:
                break
```

**The key pattern**: Keep calling the API while there are tool uses, executing them and feeding results back.

### What Claude Actually Learns

This is what makes memory powerful - **semantic pattern recognition**, not just syntax:

**Session 1: Thread-Based Web Scraper**

```python
# Bug: Race condition
class WebScraper:
    def __init__(self):
        self.results = []  # Shared state!
    
    def scrape_urls(self, urls):
        with ThreadPoolExecutor() as executor:
            for future in as_completed(futures):
                self.results.append(future.result())  # RACE!
```

**What Claude Stores in Memory** (example file: `/memories/concurrency_patterns/thread_safety.md`):

When Claude encounters this pattern, it stores the following insights to its memory files:
- **Symptom**: Inconsistent results in concurrent operations
- **Cause**: Shared mutable state (lists/dicts) modified from multiple threads
- **Solution**: Use locks, thread-safe data structures, or return results instead
- **Red flags**: Instance variables in thread callbacks, unused locks, counter increments

---

**Session 2: Async API Client** (New conversation!)

Claude checks memory FIRST, finds the thread-safety pattern, then:
1. **Recognizes** similar pattern in async code (coroutines can interleave too)
2. **Applies** the solution immediately (no re-learning needed)
3. **Explains** with reference to stored knowledge

```python
# Claude spots this immediately:
async def fetch_all(self, endpoints):
    for coro in asyncio.as_completed(tasks):
        self.responses.append(await coro)  # Same pattern!
```

---

**Why This Matters:**

- ❌ **Syntax checkers** miss race conditions entirely
- ✅ **Claude learns** architectural patterns and applies them across contexts
- ✅ **Cross-language**: Pattern applies to Go, Java, Rust concurrency too
- ✅ **Gets better**: Each review adds to the knowledge base

### Sample Code Files

The demo uses these sample files (all have concurrency/thread-safety bugs):

- `memory_demo/sample_code/web_scraper_v1.py` - Race condition: threads modifying shared state
- `memory_demo/sample_code/api_client_v1.py` - Similar concurrency bug in async context
- `memory_demo/sample_code/data_processor_v1.py` - Multiple concurrency issues for long session demo

Let's look at one:

**`memory_demo/sample_code/web_scraper_v1.py`**

```python
"""
Concurrent web scraper with a race condition bug.
Multiple threads modify shared state without synchronization.
"""

import time
from concurrent.futures import ThreadPoolExecutor, as_completed
from typing import List, Dict

import requests


class WebScraper:
    """Web scraper that fetches multiple URLs concurrently."""

    def __init__(self, max_workers: int = 10):
        self.max_workers = max_workers
        self.results = []  # BUG: Shared mutable state accessed by multiple threads!
        self.failed_urls = []  # BUG: Another race condition!

    def fetch_url(self, url: str) -> Dict[str, any]:
        """Fetch a single URL and return the result."""
        try:
            response = requests.get(url, timeout=5)
            response.raise_for_status()
            return {
                "url": url,
                "status": response.status_code,
                "content_length": len(response.content),
            }
        except requests.exceptions.RequestException as e:
            return {"url": url, "error": str(e)}

    def scrape_urls(self, urls: List[str]) -> List[Dict[str, any]]:
        """
        Scrape multiple URLs concurrently.

        BUG: self.results is accessed from multiple threads without locking!
        This causes race conditions where results can be lost or corrupted.
        """
        with ThreadPoolExecutor(max_workers=self.max_workers) as executor:
            futures = [executor.submit(self.fetch_url, url) for url in urls]

            for future in as_completed(futures):
                result = future.result()

                # RACE CONDITION: Multiple threads append to self.results simultaneously
                if "error" in result:
                    self.failed_urls.append(result["url"])  # RACE CONDITION
                else:
                    self.results.append(result)  # RACE CONDITION

        return self.results
```

**Bug**: Multiple threads modify `self.results` and `self.failed_urls` without locking!

Claude will:
1. Identify the race conditions
2. Store the pattern in `/memories/concurrency_patterns/thread_safety.md`
3. Apply this concurrency pattern to async code in Session 2

### Demo Overview

We've built a complete Code Review Assistant. The implementation is in `memory_demo/code_review_demo.py`.

**To run the interactive demo:**
```bash
python memory_demo/code_review_demo.py
```

The demo demonstrates:
1. **Session 1**: Review Python code with a bug → Claude learns the pattern
2. **Session 2**: Review similar code (new conversation) → Claude applies the pattern
3. **Session 3**: Long review session → Context editing keeps it manageable

## 7. Best Practices & Security {#best-practices}

### Memory Management

**Do:**
- ✅ Store task-relevant patterns, not conversation history
- ✅ Organize with clear directory structure
- ✅ Use descriptive file names
- ✅ Periodically review and clean up memory

**Don't:**
- ❌ Store sensitive information (passwords, API keys, PII)
- ❌ Let memory grow unbounded
- ❌ Store everything indiscriminately

### Security: Path Traversal Protection

**Critical**: Always validate paths to prevent directory traversal attacks. See `memory_tool.py` for implementation.

### Security: Memory Poisoning

**⚠️ Critical Risk**: Memory files are read back into Claude's context, making them a potential vector for prompt injection.

**Mitigation strategies:**
1. **Content Sanitization**: Filter dangerous patterns before storing
2. **Memory Scope Isolation**: Per-user/per-project isolation  
3. **Memory Auditing**: Log and scan all memory operations
4. **Prompt Engineering**: Instruct Claude to ignore instructions in memory

See `memory_tool.py` for complete security implementation and tests in `tests/`.

## Next Steps

### Resources

- **API docs**: [Claude API reference](https://docs.claude.com/en/api/messages)
- **Usage docs**: [Memory tool](https://docs.claude.com/en/docs/agents-and-tools/tool-use/memory-tool)
- **GitHub Action**: [claude-code-action](https://github.com/anthropics/claude-code-action)
- **Support**: [support.claude.com](https://support.claude.com)

### Feedback

Memory and context management are in **beta**. Share your feedback to help us improve!